# Prediction with Rank

### Question or problem definition

But : Déterminer quel joueur va gagner en fonction du rang

### Acquire training and testing data

Importer les librairies nécessaires :

In [84]:
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Acquire Data

In [85]:
train_df = pd.read_csv('input/balanced-2014-2017.csv')
test_df = pd.read_csv('input/balanced-2018.csv')
combine = [train_df, test_df]

In [86]:
train_df.head()

,match_id,date,player1_id,player2_id,player1_rank,player2_rank,player1_rank_points,player2_rank_points,player1_elo_rating,player2_elo_rating,won
0,155602,21/04/2014,4894,4022,50.0,15.0,947.0,2065.0,1867.0,2036.0,1
1,155603,21/04/2014,4659,5763,92.0,16.0,605.0,2040.0,1861.0,2112.0,0
2,155604,21/04/2014,3908,4789,102.0,24.0,569.0,1580.0,1871.0,2085.0,0
3,155605,21/04/2014,4467,4585,28.0,43.0,1315.0,1016.0,1979.0,1902.0,0
4,155202,06/03/2014,5231,4742,31.0,1.0,1205.0,14085.0,1992.0,2510.0,1


### Analyze by describing data

Which features are available in the dataset?

In [87]:
print(train_df.columns.values)

['match_id' 'date' 'player1_id' 'player2_id' 'player1_rank' 'player2_rank'
 'player1_rank_points' 'player2_rank_points' 'player1_elo_rating'
 'player2_elo_rating' 'won']


In [88]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11602 entries, 0 to 11601
Data columns (total 11 columns):
match_id               11602 non-null int64
date                   11602 non-null object
player1_id             11602 non-null int64
player2_id             11602 non-null int64
player1_rank           11507 non-null float64
player2_rank           11505 non-null float64
player1_rank_points    11507 non-null float64
player2_rank_points    11505 non-null float64
player1_elo_rating     11586 non-null float64
player2_elo_rating     11586 non-null float64
won                    11602 non-null int64
dtypes: float64(6), int64(4), object(1)
memory usage: 997.1+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974 entries, 0 to 2973
Data columns (total 11 columns):
match_id               2974 non-null int64
date                   2974 non-null object
player1_id             2974 non-null int64
player2_id             2974 non-null int64
player1_r

Features contain blank, null or empty values :
- train :
    - player1_rank
    - player2_rank
    - player1_rank_points
    - player2_rank_points
    - player1_elo_rating
    - player2_elo_rating
- test :
    - player1_rank
    - player2_rank
    - player1_rank_points
    - player2_rank_points
    - player1_elo_rating
    - player2_elo_rating

In [89]:
train_df = train_df.dropna(subset=['player1_rank_points','player2_rank_points'])
test_df = test_df.dropna(subset=['player1_rank_points','player2_rank_points'])
combine = [train_df, test_df]

In [90]:
train_df = train_df.drop(['match_id','date','player1_id','player2_id','player1_rank','player2_rank','player1_elo_rating','player2_elo_rating'], axis=1)
test_df = test_df.drop(['match_id','date','player1_id','player2_id','player1_rank','player2_rank','player1_elo_rating','player2_elo_rating'], axis=1)

In [91]:
train_df.head()

,player1_rank_points,player2_rank_points,won
0,947.0,2065.0,1
1,605.0,2040.0,0
2,569.0,1580.0,0
3,1315.0,1016.0,0
4,1205.0,14085.0,1


In [92]:
test_df.head()

,player1_rank_points,player2_rank_points,won
0,1627.0,815.0,1
1,851.0,626.0,0
2,864.0,638.0,0
3,895.0,932.0,0
4,1295.0,586.0,1


In [93]:
combine = [train_df, test_df]
for dataset in combine:
    dataset['dif_rank_points'] = dataset['player1_rank_points'] - dataset['player2_rank_points']
train_df = train_df.drop(['player1_rank_points','player2_rank_points'], axis=1)
test_df = test_df.drop(['player1_rank_points','player2_rank_points'], axis=1)
train_df.head()

,won,dif_rank_points
0,1,-1118.0
1,0,-1435.0
2,0,-1011.0
3,0,299.0
4,1,-12880.0


### Model, predict and solve

In [94]:
# machine learning
from sklearn.linear_model import LogisticRegression

In [95]:
X_train = train_df.drop("won", axis=1)
Y_train = train_df["won"]
X_test = test_df.drop('won', axis=1).copy()
Y_test = test_df['won']
X_train.shape, Y_train.shape, X_test.shape

((11423, 1), (11423,), (2947, 1))

In [96]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

C:\Users\33695\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [97]:
from sklearn.metrics import classification_report

In [98]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1510
           1       0.63      0.64      0.63      1437

    accuracy                           0.64      2947
   macro avg       0.64      0.64      0.64      2947
weighted avg       0.64      0.64      0.64      2947



In [99]:
from sklearn.metrics import confusion_matrix

In [101]:
confusion_matrix(Y_test, Y_pred)

array([[976, 534],
       [523, 914]], dtype=int64)

In [100]:
Y_test.shape

(2947,)

In [102]:
(976+914)/2947

0.6413301662707839